### 1. Transform Data from Wikipedia page

In [44]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # to send requesto url
print('Libraries imported')

Libraries imported


In [45]:
#create url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [46]:
#Request web server to connect
Toronto_url = requests.get(url)
Toronto_url

<Response [200]>

In [47]:
#read text from the url with read_html
Toronta_data = pd.read_html(Toronto_url.text)

In [48]:
#check how many tables are in the html
print('There are {} tables type of {}'.format(len(Toronta_data), type(Toronta_data)))

There are 3 tables type of <class 'list'>


In [49]:
# Take the table that with information for this project
Toronto_final = Toronta_data[0]
Toronto_final.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Rename PostalCode

In [56]:
Toronto_final.rename(columns={'Postcode':'Postalcode'}, inplace = True)
Toronto_final.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Remove rows with Borought 'Not Assigned'

In [57]:
print('Size of Toronto_final before drop', Toronto_final.shape)

IndexNames =  Toronto_final[Toronto_final['Borough']=='Not assigned'].index
Toronto_final.drop(IndexNames,inplace = True)

print('Size of Toronto_final after drop', Toronto_final.shape)

Toronto_final.head()

Size of Toronto_final before drop (288, 3)
Size of Toronto_final after drop (211, 3)


,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [69]:
test = Toronto_final.groupby('Postalcode')
test.apply()

### End of data transform

### 2. get the latitude and the longitude coordinates of each neighborhood.

In [32]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [35]:
Geospatial_df = pd.read_csv('Geospatial_data.csv')
Geospatial_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
Geospatial_df.rename(columns={'Postal Code':'Postcode'}, inplace = True)
Geospatial_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [37]:
print('size of Geospatial_df is', Geospatial_df.shape)
print('size of Toronto_final is', Toronto_final.shape)

size of Geospatial_df is (103, 3)
size of Toronto_final is (288, 3)


In [42]:
Toronto_df = pd.merge(Toronto_final, Geospatial_df, on='Postcode',how='inner')
Toronto_df.head(11)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Not assigned,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


In [43]:
Toronto_df.shape

(211, 5)

### End of latitude and the longitude

### 3. Explore and cluster the neighborhoods in Toronto